# ETL Pipeline for Pterodactyl Minecraft Servers Analysis

## Index

- Install requierements
- Import libraries and setup key variables
- Get the Uptime Kuma information
- Load data into the Postgres database

## Install requierements

In [ ]:
!pip install -r requirements.txt
!pip install uptime-kuma-api

## Import libraries and setup key variables
Remember to add you own credentials in the .env file for them to be loaded here

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from uptime_kuma_api import UptimeKumaApi, MonitorType

# Load .env file credentials
load_dotenv()

# Database connection
host = os.getenv('POSTGRES_HOST')
port = os.getenv('POSTGRES_PORT')
dbname = os.getenv('POSTGRES_DBNAME')
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
connection = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

# Kuma connection
kuma_token = os.getenv('TOKEN')
kuma_url = os.getenv('KUMA_URL')
kuma_user = os.getenv('KUMA_USER')
kuma_pass = os.getenv('KUMA_PASS')

# Connect to Kuma
api = UptimeKumaApi(kuma_url)
api.login(kuma_user, kuma_pass)

## Create monitors from Postgres information in Kuma


Create nodes monitors and put them into node's group

In [ ]:
# Create the Nodes group
existing_monitors = api.get_monitors()
monitor_name = 'Nodes'

# Check if the monitor_name already exists
if any(monitor_name == monitor.get('name') for monitor in existing_monitors):
    print(f"The monitor group '{monitor_name}' already exists.")
else:
    # Create the monitor group if it doesn't exist
    api.add_monitor(
        type=MonitorType.GROUP,
        name=monitor_name,
    )

# Looking for the 'Nodes' group id
monitor_group = 'Nodes'
existing_monitors = api.get_monitors()

# Find the monitor with the specified name and type
target_monitor = next(
    (monitor for monitor in existing_monitors if monitor.get('name') == monitor_group and monitor.get('type') == MonitorType.GROUP.value),
    None
)

# Transform the id from int to str
monitor_id = target_monitor.get('id')
monitor_id = str(monitor_id)

# Create connection with PostgreSQL and create the pairs key-values using server name and nodes links
SCHEMA = 'pterodactyl'
engine = create_engine(connection)

with engine.connect() as conn:
    result_nodes = conn.execute(text(f'select nodes.id, nodes.name, nodes.fqdn from {SCHEMA}.nodes order by nodes.id'))

result_nodes = result_nodes.fetchall()

# Modify the result_nodes dictionary creation
result_nodes_dict = {f'{node_id} | {node_name}': f'http://{node_fqdn}' for node_id, node_name, node_fqdn in result_nodes}
result_nodes_dict

for server_name, server_url in result_nodes_dict.items():
    monitor_exists = any(monitor['name'] == server_name for monitor in existing_monitors)
    if not monitor_exists:
        api.add_monitor(
            type=MonitorType.HTTP,
            name=server_name,
            url=server_url,
            accepted_statuscodes=['401'],
            parent=monitor_id
        )
# 401 unico aceptado

Create servers monitors and put them into server's group

In [ ]:
# Extract information about server name, egg name, node name and port from each server
SCHEMA = 'pterodactyl'
engine = create_engine(connection)
with engine.connect() as conn:
    result_servers = conn.execute(text(f'select servers.name as "servers_names", servers.identifier AS "servers_identifier", eggs.name as "eggs_names", nodes.name as "nodes_names", allocations.port as "port" from {SCHEMA}.servers join {SCHEMA}.eggs on servers.egg_id =eggs.id join {SCHEMA}.nodes on servers.node_id = nodes.id JOIN {SCHEMA}.allocations ON allocations.id = servers.allocation_id group by servers.id, servers.name, eggs.name, nodes.name, allocations.port HAVING Servers.is_active = True order by servers.id'))
result_servers = result_servers.fetchall()
result_servers

# Create a nested dictionary
nested_dict = {}

for server_info in result_servers:
    server_name, server_identifier, server_egg, server_node, server_port = server_info

    # Create a dictionary for each server
    server_dict = {
        'identifier': server_identifier,
        'egg': server_egg,
        'node': server_node,
        'port': server_port
    }

    # Add the server dictionary to the nested dictionary with the server name as the key
    nested_dict[server_name] = server_dict

# Determine the game and add a monitor for each server in filtered_servers
for server_name, server_info in nested_dict.items():
    server_identifier = server_info['identifier']
    server_egg = server_info['egg']
    server_node = server_info['node']
    server_port = server_info['port']

    # Check if the monitor already exists
    #existing_monitors = api.get_monitors()
    #monitor_exists = any(monitor['name'] == server_node for monitor in existing_monitors)

    # Check if server_node has been added
    #if not monitor_exists:
        #api.add_monitor(
            #type=MonitorType.GROUP,
            #name=server_info['node'],
        #)

    # Extract only the first word of 'node' in lowercase
    node_lowercase = server_node.split()[0].lower()

    # Determine the game based on the server egg
    if 'Bedrock' in server_egg:
        game_name = 'minecraftbe'
    else:
        game_name = 'minecraft'

    # Create the host by concatenating the first word with '.stiv.tech'
    host_minecraft = f"{node_lowercase}.stiv.tech"

    port_minecraft = server_port

    # Get the list of monitors
    monitors = api.get_monitors()

    # Process host_minecraft for comparison
    processed_host = host_minecraft.split('.')[0]

    # Iterate through monitors to find a match based on processed host
    for monitor in monitors:
        if monitor['type'] == "<MonitorType.GROUP: 'group'>":  # Skip non-group monitors
            continue
        
        print(monitor['name']
              )
        # Process monitor name for comparison
        words = monitor['name'].lower().split(' ')
        if len(words) >= 2:
            processed_name = words[-2]  # Use [-2] to get the second-to-last word
            print(processed_name)
        
            # Check if the processed names are similar
            if processed_host in processed_name or processed_name in processed_host:
                parent_id = monitor['id']
                break  # Stop iterating once a match is found

    # Check if the monitor already exists
    existing_monitors = api.get_monitors()
    monitor_exists = any(monitor['name'] == server_identifier + ' | ' + server_name for monitor in existing_monitors)

     # Add the monitor only if it doesn't exist
    if not monitor_exists:
        api.add_monitor(
            type=MonitorType.GAMEDIG,
            name=server_identifier + ' | ' + server_name,
            hostname=host_minecraft,
            port=port_minecraft,
            game=game_name,
            parent= parent_id
        )